In [ ]:
##### This code is from Updated Testing Reddit - No Con- bias (Fictitious Play)-01092022
##### This code replace the big real datanetwork with small sythetic network
import scipy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import pandas as pd
import scipy.io
import collections
import sys
import scipy.optimize
import os
import os.path
from math import comb
from game import Game, GameResult
from utils import *

save_path = os.getcwd()

calculate_polarization = calculate_polarization1

    # Fixed initial condition + memeory = 50

## Mathmatic Functions

### 1. Import Network

In [ ]:
network_name = 'karate'

In [ ]:
# network = import_network(network_name)

# G, s, n = network.init()
# nxG = nx.from_numpy_array(G)

# ############################ Make Innate Opinion ################################

# # Calculate Key Values & Visualization #######################################3
# # the Laplacian matrix
# L = scipy.sparse.csgraph.laplacian(G, normed=False)
# A = np.linalg.inv(np.identity(n) + L)  # A = (I + L)^(-1)  Adjacency Matrix
# # call the function to calculate the number of edges
# m = num_edges(L, n)
# columnsum_ij = np.sum(A, axis=0)
# plt.figure(figsize=(20, 20))

In [ ]:
# TODO make this a network import called 'manual'

# # #################### Import Sythetic Network Data
df = pd.read_csv (save_path+'/data/5Adj_matrix.csv', header=None)
G = np.array(df[df.columns[:]])
# # # print(G)
df1 = pd.read_csv(save_path+'/data/5innate_op.csv', header = None)
s = np.array(df1[df1.columns[:]])

# print(s)
# print(G.shape)

# # Set n according to the data
n = len(s[:])

In [ ]:
################## Process the Network Data
L = scipy.sparse.csgraph.laplacian(G, normed=False)  # Return the Laplacian matrix
A = np.linalg.inv(np.identity(n) + L)  # A = (I + L)^(-1)\n  Stanford paper theory
m = num_edges(L, n)                    # call the function to calculate the number of edges
# what the twitter graph looks like
nxG = nx.from_numpy_array(G)
#plt.figure(figsize=(20, 20))
# nx.draw(nxG).
columnsum_ij = np.sum(A, axis=0)
print(columnsum_ij)
# s = [0.5]*n
# s = np.reshape(s, (n, 1))
print(s)

### 2. Equilibrium & Polarization 
$$P(z) = z ^T * z $$


## 5. Innate Op and Game

# Fictitious Play Start !


In [ ]:
# Calculate network polarization
calculate_polarization(s, n, A, L)

In [ ]:
# Game Parameters
game_rounds = 3
memory = 10
k = 2
experiment = 1

In [ ]:
# configure the game
game = Game(s, A, L, k)
# run the game
result = game.run(game_rounds, memory)

In [ ]:
def save(result: GameResult, k, experiment):
    pd.DataFrame(result.payoff_matrix).to_csv(
        f'results/Payoff-Matrix-k-{k}-experiment-{experiment}.csv')

    # Save the original standard output
    original_stdout = sys.stdout

    with open(f'results/Result-k-{k}-experiment-{experiment}.txt', "w") as f:
        # Change the standard output to the file we created.
        sys.stdout = f

        print('Initial Condition -(agent, opinion, pol)')
        # print(f'Innate op {s}')
        # print(f'Adjacency matrix {G}')
        # print('Selected Nodeset, k_Opinions, Steady-state polarization')
        print(f'Max:\t{result.first_max}')
        print(f'Min:\t{result.first_min}')

        print('_____________________')
        print(f'Max Pol:\t{result.equi_max}')
        print(f'Min Pol:\t{result.equi_min}')

        # MAXimizer's distribution of LAST 100 iteration
        print('Max_distribution_last_100')
        max_l100_fre = result.max_history_last_100/100
        print(max_l100_fre[np.nonzero(max_l100_fre)])
        # print for small network
        # print(max_history_last_100)
        # # Print for Large Network
        print(np.nonzero(max_l100_fre))

        columns = np.nonzero(max_l100_fre)
        columns = list(columns[0])
        for column in columns:
            (k_nodes, opinions) = game.map_action(column)
            print(f'Max Nodes: {k_nodes}\tOpinion: {opinions}')

        print('Max_distribution_all')
        max_fre = result.max_history/result.game_rounds
        print(max_fre[np.nonzero(max_fre)])
        print([np.nonzero(max_fre)])

        # TODO confirm isn't this block the same as the one above
        columns_all = np.nonzero(max_l100_fre)
        columns_all = list(columns_all[0])
        for column in columns_all:
            (k_nodes, opinions) = game.map_action(column)
            print(f'Max Nodes: {k_nodes}\tOpinion: {opinions}')

        # MINimizer's Strategy in the last 100 round
        counter = collections.Counter(result.min_touched_last_100)
        # frequency of all min options in order
        fla_min_fre = np.array(list(counter.values()))/(100)
        print('Min_distribution_last_100')
        print(fla_min_fre)
        print(counter)
        # print(min_touched_last_100)

        # a dictionary include {'min_option': count of this choice}
        counter_1 = collections.Counter(result.min_touched_all)
        # frequency of all min options in order
        fla_min_fre_1 = np.array(list(counter_1.values()))/result.game_rounds
        print('Min_distribution_all')
        print(fla_min_fre_1)
        print(counter_1)
        np.set_printoptions(precision=3)

        # a dictionary include {'min_option': count of this choice}
        counter_a = collections.Counter(result.min_history)
        print(counter_a)

        print(f'min_recent_{memory}_touched')
        print(result.min_touched)
        print(f'max_recent_{memory}_touched')
        print(result.max_touched)

        # Reset the standard output to its original value
        sys.stdout = original_stdout

In [ ]:
save(result, k, experiment)